In [11]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm

from time import time, sleep
from datetime import datetime
from random import randrange

from ast import literal_eval
import numpy as np
import pandas as pd
import re
import string
import glob

import spacy
from spacy.lang.en import English
from nltk import PorterStemmer
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize 

from sklearn.feature_extraction.text import TfidfVectorizer

### Data Cleanup

General functions to load the data and carry out some basic Pandas cleaning

In [12]:
def load_data(path=r'scraped_data/base'):
    all_files = glob.glob(path + "/*.csv")
    df_list = [pd.read_csv(file, index_col=None, header=0) for file in all_files]
    return pd.concat(df_list, axis=0, ignore_index=True) 

In [13]:
def clean_df(df):
    df=df.copy()
    df.drop_duplicates(inplace=True)
    df.dropna(subset=['Salary'],inplace=True)
    df['Location']= df['Location1'].fillna(df['Location2'])
    df.drop(['Location1','Location2'],axis=1,inplace=True)
    df = df.apply(lambda x: x.str.replace('\nnew','').str.replace('\n',''))
    return df

### Title

The job title must be lemmatized and cleaned so it can be vectorized later on. Stemming was not used as this can limit interpretability

In [14]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in tokens:
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

In [15]:
def clean_title(array):
    stopwords = English.Defaults.stop_words
    array = (array
                .str.lower()
                .str.replace('[0-9£–]','',regex=True)
                .apply(word_tokenize)
                .apply(lambda x: [i for i in x if i not in stopwords])
                .apply(lambda x: [i for i in x if len(i) >1])
                .apply(pos_tag)
                .apply(lemmatize_sentence)
                .apply(' '.join)
                .apply(lambda x: x.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))))
            )
    return array

### Company

In [16]:
def list_df(x,y):
    x = literal_eval(x)
    y = literal_eval(y)
    k = dict(zip(x,y))
    return k

In [17]:
def clean_company_data(df):
    company_info = df.copy()
    company_info.columns = ['Company','heading','value']
    company_info2 = pd.DataFrame(list(company_info.apply(lambda x: list_df(x.heading,x.value),axis=1)))
    company_info = pd.merge(company_info,company_info2, left_index=True, right_index=True)
    company_info.drop(['heading','value','Website','Headquarters'],axis=1,inplace=True)    
    company_info.fillna('no_data',inplace=True)
    company_info = company_info.sort_values(by='Employees')
    company_info['Company'] = company_info['Company'].str.replace('+',' ')
    company_info.drop_duplicates(subset='Company',inplace=True)
    return company_info

### Salary

In [18]:
def clean_salary(df,col):
    df=df.copy()
    # pull salary information if in title
    reg1 = '(?<=£)([0-9,-k]+)'
    df['TitleSalary'] = df['Title'].apply(lambda x: ' '.join(re.findall(reg1,x)))
    df['TitleSalary'] = df['TitleSalary'].replace(r'^\s*$', np.nan, regex=True)
    df[col] = df[col].fillna(df['TitleSalary'])
    df.drop('TitleSalary',axis=1,inplace=True)   
    
    reg2 = r'(?<=£)([0-9,]+)'
    df['min_salary'] = df[col].apply(lambda x: re.findall(reg2,x)[0].replace(',','')).astype(float)
    df['max_salary'] = df[col].apply(lambda x: re.findall(reg2,x)[-1].replace(',','')).astype(float)
    
    reg3 = r'([A-z]+)'
    df['salary_period'] = df[col].apply(lambda x: re.findall(reg3,x)[1])
    df.drop([col],axis=1,inplace=True)
    
    # Turning hourly/daily salaries into yearly. In the end only yearly salaries were used
    
    salary_period_calc = {
                            'hour':2000, # 40 hour weeks, 50 weeks
                            'year': 1,   
                            'day': 240,  # 240 working days per year
                            'month': 12, 
                            'week': 50   # 50 working weeks
                            }  
    
    df['min_salary'] = df['min_salary'] * df['salary_period'].map(salary_period_calc)
    df['max_salary'] = df['max_salary'] * df['salary_period'].map(salary_period_calc)
    df['mid_salary'] = (df['min_salary']+df['max_salary'])/2    
    return df

### Location

In [19]:
def clean_location(array):
    array[array.str.contains('London')] = 'London' # This causes setting with copy warning
    array = (array
             .str.replace('[A-Z]{1,2}\d[A-Z\d]? ?\d[A-Z]{2}','',regex=True) # remove postcodes
             .str.replace(r'[A-Z]{1,2}\d[A-Z\d]?','',regex=True) # remove postcoddes
             .apply(lambda x: x.strip())
             .str.replace(r'[^\w\s]','',regex=True)
             .str.replace(' ','_')         
            )
    return array

### Full Descriptions

In [20]:
def remove_dupes(df):
    df = df.copy()
    df.columns = ['url','description']
    df['len'] = df['description'].apply(len)
    df = df.sort_values(by='len',ascending=False)
    df.drop_duplicates(subset = 'url',inplace=True)
    df.drop('len',axis=1,inplace=True)
    return df

In [22]:
# Attempting to get years experience from the job description. This didn't provide much data and wasn't used.

def get_years(array):
    X = array
    reg = r'\d*\s(?=years e)'
    X = X.apply(lambda x: x.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))))
    return X.apply(lambda x: re.findall(reg,x)).apply(lambda x: clean_years(x))

In [21]:
def clean_years(years):
    if len(years)>0:
        return years[0].strip()
    else:
        return np.nan

In [23]:
# This function is similar to the job title function but with some subtle differences due to formatting.

def clean_text(array):
    stopwords = English.Defaults.stop_words   
    array=(array
            .apply(lambda x: ' '.join(x.split('\\n')))
            .str.lower()
            .str.replace('[0-9£–]','',regex=True)
            .apply(word_tokenize)
            .apply(lambda x: [i for i in x if i not in stopwords])
            .apply(pos_tag)
            .apply(lemmatize_sentence)
            .apply(' '.join)
            .apply(lambda x: x.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))))
            .str.strip()) 
    return array

### Cleaning

In [24]:
def translator(text, conversion_dict, before=None):
    """
    Translate words from a text using a conversion dictionary

    Arguments:
        text: the text to be translated
        conversion_dict: the conversion dictionary
        before: a function to transform the input
        (by default it will to a lowercase)
    """
    # if empty:
    if not text: return text
    # preliminary transformation:
    before = before or str.lower
    t = before(text)
    for key, value in conversion_dict.items():
        t = t.replace(key, value)
    return t

In [25]:
# for EDA

def job_cat(x):
    if 'analys' in x:
        return 'Analyst'
    elif 'scien' in x:
        return 'Scientist'
    elif 'enginee' in x:
        return 'Engineer'
    else:
        return 'Other'

In [26]:
def clean_merged(df):
    
    replacements = {
    'team work:':'teamwork',
    'power bi':'powerbi',
    "' s'":' ',
    " s ":' ',
    "''":' ',
    '’':' ',
    'big query':'bigquery'} # manual replacements
    
    df=df.copy()
    df['description'] = df['description'].str.strip().replace('',np.nan).fillna(df['Summary'])
    df.drop(['url','Summary','min_salary','max_salary','Company'],axis=1,inplace=True)
    df.fillna('no_data',inplace=True)
    df.drop_duplicates(inplace=True)

    df['description'] = df['description'].apply(lambda x: translator(x,replacements))
    df['title_category'] = df['Title'].apply(lambda x: job_cat(x))
    return df

In [27]:
# Outliers removed -  low salaries which may be internships or incorrect data 
# There are very few salaries over 200k and this is the region where there is much more variance.

def remove_outliers(df,col):
    return df[(df[col]>16000) & (df[col]<200000)]  # Outliers removed 

### Apply Cleanup Functions

In [28]:
# Load scraped jobs data

df = load_data()
df = clean_df(df)

In [29]:
# Load scraped company data

company_info = load_data(path=r'scraped_data/companies')
company_info = clean_company_data(company_info)

In [30]:
# Load & clean full descriptions

full_desc = load_data(path=r'scraped_data/descriptions')
full_desc = remove_dupes(full_desc)
full_desc['years_exp'] = get_years(full_desc['description'])
full_desc['description'] = clean_text(full_desc['description'])

In [31]:
# Clean jobs data

df['Title'] = clean_title(df['Title'])
df = clean_salary(df,'Salary')
df['Location'] = clean_location(df['Location'])
df = remove_outliers(df,'mid_salary')

<ipython-input-19-062a9a7b5460>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  array[array.str.contains('London')] = 'London'


In [32]:
# Merge scraped data

df = pd.merge(df,company_info,left_on='Company',right_on='Company',how='left')
df = pd.merge(df,full_desc,left_on='url',right_on='url')
df = clean_merged(df)
df.head()

,Title,Location,salary_period,mid_salary,Employees,Industry,Revenue,description,years_exp,title_category
0,high statistical data scientist g,London,year,28996.0,no_data,no_data,no_data,role require apply knowledge statistic program...,no_data,Scientist
2,senior statistical data scientist g,London,year,34090.0,no_data,no_data,no_data,role require apply knowledge statistic program...,no_data,Scientist
4,data scientist,London,year,65000.0,51 to 200,Consulting and Business Services,less than £1m,data scientist london united kingdom salary ...,no_data,Scientist
6,data scientist,London,year,67500.0,51 to 200,Consulting and Business Services,less than £1m,data scientist london united kingdom salary ...,no_data,Scientist
8,data scientist,London,day,132000.0,51 to 200,Consulting and Business Services,less than £1m,data scientist day remote london base month ch...,no_data,Scientist


In [33]:
df.to_csv('jobs_desc.csv',index=False)